In [74]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [82]:
def obtener_detalles_inmueble(url_inmueble):
    try:
        response = requests.get(url_inmueble)
        soup = BeautifulSoup(response.text, "html.parser")
        
        nombre = soup.find("h1", class_="ad-title")
        nombre = nombre.text.strip() if nombre else "Error al acceder al inmueble"
        # Buscar el código postal
        #ubicacion = soup.find("div", class_="title-location")
        #ubicacion = ubicacion.text.strip() if ubicacion else "Ubicación no disponible"
        #codigo_postal = re.search(r'\b28\d{3}\b', ubicacion)  # Busca códigos postales que comienzan con '28'
        #codigo_postal = codigo_postal.group() if codigo_postal else "Código postal no disponible"


        # Buscar el nombre de la agencia
        agencia = soup.find("p", class_="owner-info__name")
        agencia = agencia.find("a").text.strip() if agencia and agencia.find("a") else "Agencia no disponible"

        # Buscar la lista de características y extraer solo el precio por metro cuadrado
        features_summary = soup.find("ul", class_="features-summary")
        precio_m2 = None
        if features_summary:
            items = features_summary.find_all("li", class_="features-summary__item")
            for item in items:
                if "€/m²" in item.text:
                    precio_m2 = item.text.strip()
                    break
        precio_m2 = precio_m2 if precio_m2 else "Error al procesar precio m²"
        
        # Otros detalles (precio, superficie, etc.) como antes
        precio = soup.find("div", class_="price__value jsPriceValue")
        precio = precio.text.strip() if precio else "Error al procesar precio"
        
        superficie = soup.find("span", class_="features__value")
        superficie = superficie.text.strip() if superficie else "Error al procesar superficie"
        
        actualizacion = soup.find("div", class_="details__block last-update")
        actualizacion = actualizacion.text.strip().replace("Última actualización\n", "").strip() if actualizacion else "Error al procesar actualizacion"

        consumo = soup.find_all("span", class_="energy-certificate__tag")
        consumo_etiquetas = []
        for etiqueta in consumo:
            if 'energy-certificate__tag--' in etiqueta['class'][1]:
                letra_consumo = etiqueta['class'][1].split('--')[1].strip()
                if letra_consumo in ["a", "b", "c", "d", "e", "f", "g"]: 
                    consumo_etiquetas.append(letra_consumo.upper())
        consumo = ', '.join(consumo_etiquetas) if consumo_etiquetas else "Error al procesar consumo"

        emisiones = soup.find_all("span", class_="energy-certificate__tag")
        emisiones_etiquetas = []
        for etiqueta in emisiones:
            if 'energy-certificate__tag--' in etiqueta['class'][1]:
                letra_emisiones = etiqueta['class'][1].split('--')[1].strip()
                if letra_emisiones in ["a", "b", "c", "d", "e", "f", "g"]:
                    emisiones_etiquetas.append(letra_emisiones.upper())
        emisiones = ', '.join(emisiones_etiquetas) if emisiones_etiquetas else "Error al procesar emisiones"

        span_municipio = soup.find("span", id="gaCusVar")
        if span_municipio:
            data_var = span_municipio.get("data-var", "")
            match = re.search(r"M(\d{5})", data_var)  # Busca un código postal con M + 5 dígitos
            codigo_postal = match.group(1) if match else "Código postal no disponible"
        else:
            codigo_postal = "Código postal no disponible"

        # Características adicionales
        caracteristicas = soup.find("div", class_="features-container")
        detalles = {}
        if caracteristicas:
            secciones = caracteristicas.find_all("div", class_="features__content")
            for seccion in secciones:
                features = seccion.find_all("div", class_="features__feature")
                for feature in features:
                    label = feature.find("span", class_="features__label").text.strip().replace(":", "")
                    value = feature.find("span", class_="features__value").text.strip() if feature.find("span", class_="features__value") else "N/A"
                    detalles[label] = value
        return nombre, agencia, precio_m2, precio, superficie, actualizacion, consumo, emisiones, codigo_postal, detalles
    except Exception as e:
        return "Error al acceder al inmueble", "Error al acceder al inmueble", "Error al acceder al inmueble", "Error al acceder al inmueble", "Error al acceder al inmueble", "Error al acceder al inmueble", "Error al acceder al inmueble", "Error al acceder al inmueble", "Error al acceder al inmueble", {}

def obtener_inmuebles_paginas(base_url, paginas=2):
    todos_los_inmuebles = []
    for i in range(1, paginas + 1):
        print(f"Procesando página {i}")
        url_pagina = f"{base_url}{i}/"
        response = requests.get(url_pagina)
        soup = BeautifulSoup(response.text, "html.parser")
        titulos_soup = soup.find_all("a", class_="ad-preview__title")
        for titulo_soup in titulos_soup:
            nombre = titulo_soup.text.strip()
            href = "https://www.pisos.com" + titulo_soup['href']
            nombre_detalle, agencia, precio_m2, precio, superficie, actualizacion, consumo, emisiones, codigo_postal, detalles = obtener_detalles_inmueble(href)
            inmueble = {
                "nombre": nombre,
                "agencia": agencia,  # Aquí se agrega el nombre de la agencia
                "precio_m2": precio_m2,  # Cambié aquí 'descripcion' por 'precio_m2'
                "precio": precio,
                "superficie": superficie,
                "codigo_postal": codigo_postal,
                "href": href,
                "actualizacion": actualizacion,
                "consumo": consumo,
                "emisiones": emisiones
            }
            inmueble.update(detalles)
            todos_los_inmuebles.append(inmueble)
        time.sleep(2)
    df_alquileres = pd.DataFrame(todos_los_inmuebles)
    return df_alquileres

base_url = "https://www.pisos.com/venta/pisos-madrid/"
df_alquileres = obtener_inmuebles_paginas(base_url, paginas=2)#hay 80 pags
#df_alquileres['actualizacion'] = pd.to_datetime(df_alquileres['actualizacion'].str.extract(r'(\d{2}/\d{2}/\d{4})')[0], format='%d/%m/%Y')

Procesando página 1
Procesando página 2


In [76]:
def extract_madrid_postal_code(url):
    match = re.search(r'-(28\d{3})-', url)  # Busca códigos postales que empiecen por "28"
    return match.group(1) if match else None

# Aplicar la función a la columna 'href'
df_alquileres['codigo_postal'] = df_alquileres['href'].apply(extract_madrid_postal_code)

# Mostrar el dataframe actualizado


In [84]:
df_alquileres

,nombre,agencia,precio_m2,precio,superficie,codigo_postal,href,actualizacion,consumo,emisiones,...,Teléfono,Chimenea,Urbanizado,Balcón,Carpintería interior,Gas,Adaptado a personas con movilidad reducida,Calle alumbrada,Calle asfaltada,Se aceptan mascotas
0,Casa adosada en calle de los Hermanos San Romá...,GILMAR-Madrid,3.942 €/m²,1.380.000 €,350 m²,28115,https://www.pisos.com/comprar/casa_adosada-est...,Anuncio actualizado el 17/02/2025,Error al procesar consumo,Error al procesar emisiones,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apartamento en calle del Pico de la Maliciosa,EXCLUSIVAS SANTO MAURO,3.112 €/m²,249.000 €,80 m²,28047,https://www.pisos.com/comprar/apartamento-parq...,Anuncio actualizado el 29/01/2025,Error al procesar consumo,Error al procesar emisiones,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Casa pareada en Villalbilla,DOMUM SOLUCIONES INMOBILIARIAS,2.957 €/m²,499.900 €,169 m²,28172,https://www.pisos.com/comprar/casa_pareada-vil...,Anuncio actualizado el 01/02/2025,Error al procesar consumo,Error al procesar emisiones,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Piso en calle de Nuestra Señora de Guadalupe,AGIL INMOBILIARIA,2.820 €/m²,189.000 €,67 m²,28074,https://www.pisos.com/comprar/piso-leganes_cen...,Anuncio actualizado el 03/12/2024,Error al procesar consumo,Error al procesar emisiones,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Piso en calle de la Plata, s/n","PREMIER ESPAÑA, S.A.",3.521 €/m²,309.900 €,88 m²,28148,https://www.pisos.com/comprar/piso-zona_surest...,Anuncio actualizado el 03/02/2025,Error al procesar consumo,Error al procesar emisiones,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Piso en Espartales,Estudio Alcala Espartales Sl,2.007 €/m²,263.000 €,131 m²,28005,https://www.pisos.com/comprar/piso-espartales2...,Anuncio actualizado el 15/02/2025,Error al procesar consumo,Error al procesar emisiones,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,"Chalet en Avenida de La Cabrera, 27",BREM GLOBAL INMOBILIARIA,1.350 €/m²,459.000 €,340 m²,28030,https://www.pisos.com/comprar/chalet-la_cabrer...,Anuncio actualizado el 17/02/2025,Error al procesar consumo,Error al procesar emisiones,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,Piso en calle de Vélez Málaga,Black House Servicios Inmobiliarios,3.081 €/m²,188.000 €,61 m²,28079,https://www.pisos.com/comprar/piso-palomeras_s...,Anuncio actualizado el 14/02/2025,Error al procesar consumo,Error al procesar emisiones,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,Chalet en Fuencarral - El Pardo - Fuentelarreina,JAIME VALCARCE CONSULTING INMOBILAIRIO,3.960 €/m²,2.990.000 €,755 m²,28079,https://www.pisos.com/comprar/chalet-fuentelar...,Anuncio actualizado el 17/02/2025,Error al procesar consumo,Error al procesar emisiones,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
df_alquileres["codigo_postal"].isna()

0     True
1     True
2     True
3     True
4     True
      ... 
57    True
58    True
59    True
60    True
61    True
Name: codigo_postal, Length: 62, dtype: bool

In [47]:
print(f"Ahora tenemos un DF de {df_alquileres.shape} dimensiones")
print(f"Las columnas del DF son:\n{df_alquileres.columns}")
print(f"El porcentaje de Nans por columna es: \n{df_alquileres.isna().mean().round(2)*100}")

Ahora tenemos un DF de (62, 10) dimensiones
Las columnas del DF son:
Index(['nombre', 'agencia', 'precio_m2', 'precio', 'superficie', 'href',
       'codigo_postal', 'actualizacion', 'consumo', 'emisiones'],
      dtype='object')
El porcentaje de Nans por columna es: 
nombre           0.0
agencia          0.0
precio_m2        0.0
precio           0.0
superficie       0.0
href             0.0
codigo_postal    0.0
actualizacion    0.0
consumo          0.0
emisiones        0.0
dtype: float64


In [ ]:
d

In [ ]:
df_alquileres.to_csv('scrapeoalquiler80pagsmadrid17feb.csv', index=False)